In [14]:
import tensorflow as tf
import os
import random
AUTO = tf.data.experimental.AUTOTUNE
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

In [2]:
#Detect hardware
try:
  tpu_address = "grpc://" + os.environ['COLAB_TPU_ADDR']
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu)
  print("Running on TPU ", tpu.cluster_spec().as_dict()['worker'])
  print("Number of accelerator ", strategy.num_replicas_in_sync)
except ValueError:
  print("TPU failed to initialize")

INFO:tensorflow:Initializing the TPU system: grpc://10.88.214.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.88.214.234:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU  ['10.88.214.234:8470']
Number of accelerator  8


In [71]:
SIZE = 224
IMAGE_SIZE = [SIZE, SIZE]
GCS_PATTERN = 'gs://flowers-public/tfrecords-jpeg-{}x{}/*.tfrec'.format(IMAGE_SIZE[0], IMAGE_SIZE[1])
BATCH_SIZE = 128
filenames = tf.io.gfile.glob(GCS_PATTERN)
random.shuffle(filenames)
VALIDATION_SPLIT = 0.2
split = int(len(filenames) * VALIDATION_SPLIT)
training_filenames = filenames[split:]
testing_filenames = filenames[:split]
CLASSES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
def read_tfrecord(example):
  features = {
      "image": tf.io.FixedLenFeature([], tf.string),
      "class": tf.io.FixedLenFeature([], tf.int64),
      "one_hot_class": tf.io.VarLenFeature(tf.float32)
  }
  example = tf.io.parse_single_example(example, features)
  image = example['image']
  class_label = example['class']
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [224, 224])
  image = tf.cast(image, tf.float32) / 255.0
  class_label = tf.cast(class_label, tf.int32)
  return image, class_label

def load_dataset(filenames):
  option_no_order = tf.data.Options()
  option_no_order.experimental_deterministic=False

  dataset = tf.data.Dataset.from_tensor_slices(filenames)
  dataset = dataset.with_options(option_no_order)
  dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=16, num_parallel_calls=AUTO)
  dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
  return dataset

def get_batched_dataset(filenames):
  dataset =  load_dataset(filenames)
  dataset = dataset.shuffle(2048)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=False)
  dataset = dataset.prefetch(AUTO)
  return dataset

#Distribute dataset using strategy

def get_training_data(filenames):
  dataset = get_batched_dataset(training_filenames)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

def get_testing_data(filenames):
  dataset = get_batched_dataset(testing_filenames)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

In [77]:
'''
(train_image, train_label), (test_image, test_label) = fashion_mnist.load_data()
train_image = train_image / 255.0
train_image = np.expand_dims(train_image, axis=-1)
test_image = test_image/255.0
test_image = np.expand_dims(test_image, axis=-1)
train_label = tf.keras.utils.to_categorical(train_label, 10)
test_label = tf.keras.utils.to_categorical(test_label, 10)
batch_size = 128
#Train image
train_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))
train_dataset = train_dataset.shuffle(20180)
train_dataset = train_dataset.batch(batch_size*strategy.num_replicas_in_sync, drop_remainder=True)
train_dataset = train_dataset.prefetch(AUTO)
train_dataset = strategy.experimental_distribute_dataset(train_dataset)
#Test image
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_label))
test_dataset = test_dataset.batch(batch_size*strategy.num_replicas_in_sync, drop_remainder=True)
test_dataset = test_dataset.prefetch(AUTO)
test_dataset = strategy.experimental_distribute_dataset(test_dataset)
'''

'\n(train_image, train_label), (test_image, test_label) = fashion_mnist.load_data()\ntrain_image = train_image / 255.0\ntrain_image = np.expand_dims(train_image, axis=-1)\ntest_image = test_image/255.0\ntest_image = np.expand_dims(test_image, axis=-1)\ntrain_label = tf.keras.utils.to_categorical(train_label, 10)\ntest_label = tf.keras.utils.to_categorical(test_label, 10)\nbatch_size = 128\n#Train image\ntrain_dataset = tf.data.Dataset.from_tensor_slices((train_image, train_label))\ntrain_dataset = train_dataset.shuffle(20180)\ntrain_dataset = train_dataset.batch(batch_size*strategy.num_replicas_in_sync, drop_remainder=True)\ntrain_dataset = train_dataset.prefetch(AUTO)\ntrain_dataset = strategy.experimental_distribute_dataset(train_dataset)\n#Test image\ntest_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_label))\ntest_dataset = test_dataset.batch(batch_size*strategy.num_replicas_in_sync, drop_remainder=True)\ntest_dataset = test_dataset.prefetch(AUTO)\ntest_dataset = s

## Define custome model

In [72]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3))
    self.maxpool1 = tf.keras.layers.MaxPool2D(2,2)
    self.conv2 = tf.keras.layers.Conv2D(64, (3,3), activation='relu')
    self.maxpool2 = tf.keras.layers.MaxPool2D(2,2)
    self.flatten = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(128, activation='relu')
    self.out = tf.keras.layers.Dense(10, activation='softmax')
  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.maxpool1(x)
    x = self.conv2(x)
    x = self.maxpool2(x)
    x = self.flatten(x)
    x = self.dense1(x)
    x = self.out(x)
    return x

In [75]:
with strategy.scope():
  model = MyModel()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
  train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='training_accuracy')
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='testing_accuracy')
  optimizer = tf.keras.optimizers.Adam()

  def compute_loss(labels, predictions):
    per_example_loss = loss_object(labels, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size=batch_size*strategy.num_replicas_in_sync)
  test_loss = tf.keras.metrics.Mean(name='test_loss')

  #Define train function
  
  #Distribute train and test steps
  @tf.function
  def distributed_train_step(dataset_inputs):
    per_replica_loss = strategy.run(train_steps, args=(dataset_inputs,))
    print(per_replica_loss)
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_loss, axis=None)
  @tf.function
  def distributed_test_steps(dataset_inputs):
    strategy.run(test_steps, args=(dataset_inputs,))

  def train_steps(inputs):
    image, labels = inputs
    with tf.GradientTape() as tape:
      predictions = model(image)
      loss = compute_loss(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_accuracy.update_state(labels, predictions)
    return loss
  def test_steps(inputs):
    image, labels = inputs
    predictions = model(image)
    loss = loss_object(labels, predictions)

    test_accuracy.update_state(labels, predictions)
    test_loss.update_state(loss)
    return loss

In [76]:
EPOCHS = 10
with strategy.scope():
  for epoch in range(EPOCHS):
    #Train Loop
    total_loss = 0.0
    num_batches = 0
    for x in get_training_data(training_filenames):
      total_loss += distributed_train_step(x)
      num_batches += 1
    train_loss = total_loss / num_batches

    #Testing Loop
    for x in get_testing_data(testing_filenames):
      distributed_test_steps(x)
    print(train_loss)

    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}
PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}
PerReplica:{